In [1]:
import pandas as pd
import numpy as np
from User import User
from collections import deque

In [2]:
usuario = User("historico_2022004897_murilo.pdf")

**Inicializacao**

Pegar tudo que for repetir muito no codigo e inicializar aqui junto

In [3]:
# pega todas as disciplinas Obrigatorias e optativas
disciplinas_obrigatorias = pd.read_csv(
    "materias_obrigatorias_sin.csv", encoding="UTF-8", sep=";"
)
disciplinas_optativas = pd.read_csv(
    "materias_optativas_sin.csv", encoding="UTF-8", sep=";"
)

# Combinação de disciplinas obrigatórias e optativas em um DataFrame
disciplinas = pd.concat(
    [disciplinas_obrigatorias, disciplinas_optativas], ignore_index=True
)

# Pega o dataset, ordena por periodo e transforma em um dicionario
Disciplinas_Semestre = (
    disciplinas_obrigatorias.groupby("PER.")["CODIGO"].apply(list).to_dict()
)


# Cria uma lista de disciplinas que foram cursadas
disciplinas_aprovadas = usuario.materias_aprovadas
# retira as materias que estão sendo cursadas, exclui as que fui reprovado e as que estou matriculado.

# Cria uma lista com as materias não cursadas
Disciplinas_Nao_Cursadas = disciplinas_obrigatorias["CODIGO"]
Disciplinas_Nao_Cursadas = [
    materia
    for materia in Disciplinas_Nao_Cursadas
    if materia not in disciplinas_aprovadas
]

In [4]:
def getDicionarioEquivalente(df):
    dicionario = {}
    for _, mat in df.iterrows():
        if mat["EQUIVALENTES"] != "-":
            separar = mat["EQUIVALENTES"].split(", ")
            for equiva in separar:
                dicionario[equiva] = mat["CODIGO"]
    return dicionario


teste = getDicionarioEquivalente(disciplinas)

In [5]:
def get_concluidas(materias, materia_concluidas):
    equivalencia = getDicionarioEquivalente(materias)
    concluidas = []
    for materia in materia_concluidas:
        if materia in equivalencia:
            concluidas.append(equivalencia[materia])
        else:
            concluidas.append(materia)
    return concluidas

**FUNÇÃO QUE CALCULA QUANTAS MATÉRIAS SÃO LIBERADAS E RETRONA A TABELA ORDENADA** \
Cria uma coluna que nos diz quantas materias são liberadas quando passamos na materia em questão

In [6]:
def calcularLiberam(Disciplinas_Requerimento):
    # Cria um dicionario com o index sendo CODIGO da materia e os seus pre-requisitos como valores
    requisitos = {
        codigo: pre_requisitos
        for codigo, pre_requisitos in zip(
            Disciplinas_Requerimento["CODIGO"],
            Disciplinas_Requerimento["PRE-REQUISITOS"],
        )
    }
    # Gera um vetor com todas as vezes que uma materia esta presente no Pre-requisito
    liberam = []
    for materias in requisitos:
        if requisitos[materias] != "-":
            separar = requisitos[materias].split(", ")
            for mat in separar:
                liberam.append(mat)

    # Cria a coluna de quantas materias uma disciplina libera quando concluida
    Disciplinas_Requerimento["Liberam"] = 0

    # Faz a soma das materias de acordo com quantas vezes ela aparece no vetor
    for materias in liberam:
        Disciplinas_Requerimento.loc[
            Disciplinas_Requerimento["CODIGO"] == materias, "Liberam"
        ] += 1

    # Ordena de modo que as disciplinas com mais 'Ligações' estejam acima
    # Fomatacao do dataframe [index, CODIGO,	DISCIPLINA,	PER.,	CH,	PRE-REQUISITOS,	Liberam]
    Disciplinas_Requerimento = Disciplinas_Requerimento.sort_values(
        by="Liberam", ascending=False
    )
    return Disciplinas_Requerimento


liberam = calcularLiberam(disciplinas_obrigatorias)
liberam.head()

,CODIGO,DISCIPLINA,PER.,CH,PRE-REQUISITOS,EQUIVALENTES,Liberam
14,STCO01,Algoritmos e Programação I,2,64,XDES01,"COM111, ECOP02, ECOP12",3
0,XDES01,Fundamentos de Programação,1,64,-,"COM110, ECOP11A, ECOP01, ECOP11",2
2,XDES03,Programação Web,3,64,XDES02,"COM222, ECOX05",2
3,XDES04,Engenharia de Software I,2,64,-,"COM210, ECOT01A, ECOT11A, ECOT0, ECOT11",2
18,XMAC02,Métodos Matemáticos para Análise de Dados,4,64,"MAT00A, XMAC01, STCO01",MAT013,2


**Substituir Equivalentes**

In [11]:
def substituir_equivalentes(
    disciplinas_Cursadas, disciplinas_optativas, disciplinas_obrigatorias
):
    # Criar dicionários para equivalentes
    optativas_equivalentes = {}
    obrigatorias_equivalentes = {}

    # para cada linha nas optativas
    for _, linha in disciplinas_optativas.iterrows():
        codigo = linha["CODIGO"]  # codigo recebe o codigo da materia
        # equivalentes recebe as materias equivalentes
        equivalentes = linha["EQUIVALENTES"].split(", ")
        for equivalente in equivalentes:  # para cada equivalente em equivalentes
            # obtativas_equivalentes[equivalente] vai receber o codigo
            optativas_equivalentes[equivalente] = codigo

    # para cada linha nas obrigatórias
    for _, linha in disciplinas_obrigatorias.iterrows():
        codigo = linha["CODIGO"]  # codigo recebe o codigo da materia
        # equivalentes recebe as materias equivalentes
        equivalentes = linha["EQUIVALENTES"].split(", ")
        for equivalente in equivalentes:  # para cada equivalente em equivalentes
            # obtativas_equivalentes[equivalente] vai receber o codigo
            obrigatorias_equivalentes[equivalente] = codigo

    # para um i ate o tamanho de disciplinas cursadas
    for i in range(len(disciplinas_Cursadas)):
        # codigo vai receber o valor de disciplinas_cursadas[i]
        codigo = disciplinas_Cursadas[i]

        if (codigo in list(optativas_equivalentes)):  # se tiver o codigo em optativas_equivalentes
            # troco o valor da disciplinas_cursadas[i] pelo codigo da equivalente (optativa)
            disciplinas_Cursadas[i] = optativas_equivalentes[codigo]

        # se codigo estiver em optativas_equivalentes
        elif (codigo in list(obrigatorias_equivalentes)):
            # troco o valor da disciplinas_cursadas[i] pelo codigo da equivalente (obrigatorio)
            disciplinas_Cursadas[i] = obrigatorias_equivalentes[codigo]


# Chamada da função para substituir equivalentes na lista de disciplinas cursadas
substituir_equivalentes(
    usuario.materias_aprovadas, disciplinas_optativas, disciplinas_obrigatorias
)

    CODIGO                                   DISCIPLINA     PER.  CH  \
0   XDES08                      Arquitetura de Software      7,9  64   
1   XDES09                           Padrões de Projeto      6,8  64   
2   XDES10          Engenharia de Software Experimental      6,8  64   
3   XDES11                             Tópicos em DES I      7,9  64   
4   XDES12                            Tópicos em DES II      7,9  64   
5   XDES13                     Desenvolvimento de Jogos      6,8  64   
6   XDES14     Desenvolvimento para Dispositivos Móveis      7,9  64   
7   XDES15                     Reutilização de Software      7,9  64   
8   ECOX21                    Maratona de Programação I      6,8  48   
9   ECOX22                   Maratona de Programação II      7,9  48   
10  XPAD04                        Bancos de Dados NoSQL      6,8  64   
11  SPAD05                Análise de Dados Geoespaciais      7,9  64   
12  SPAD06                           Mineração de Dados      6,8

**Recomendação Obrigatorias e Optativas**

**Estado da arte:** \
      testar com alunos que repetiram uma materia - OK \
      testar com CCO - Ok \
      Equivalencias - Dataset ok, falta fazer rodar no codigo \
      Pre-requisitos - fazer com que foque o prerequisito antes da materia propriamente dita \
      Fazer com que o indice influencie na recomendação \
      ser feliz - daqui 1 mes \
      comprar casa propria 

In [ ]:
def recomendaOptativas(cursadas, optativas):
    APH = ["XAHC06", "IEPG21", "ADM08H",
           "ADM52H", "ADM54H", "ADM58H", "ADM51H"]
    if any(set(APH).intersection(cursadas)):
        for optativa in optativas:
            return optativa
    else:
        for optativa in optativas:
            if optativa in APH:
                return optativa


Disciplinas_Requerimento = calcularLiberam(disciplinas_obrigatorias)
Disciplinas_Requerimento.head()

,CODIGO,DISCIPLINA,PER.,CH,PRE-REQUISITOS,EQUIVALENTES,Liberam
14,STCO01,Algoritmos e Programação I,2,64,XDES01,"COM111, ECOP02, ECOP12",3
0,XDES01,Fundamentos de Programação,1,64,-,"COM110, ECOP11A, ECOP01, ECOP11",2
2,XDES03,Programação Web,3,64,XDES02,"COM222, ECOX05",2
3,XDES04,Engenharia de Software I,2,64,-,"COM210, ECOT01A, ECOT11A, ECOT0, ECOT11",2
18,XMAC02,Métodos Matemáticos para Análise de Dados,4,64,"MAT00A, XMAC01, STCO01",MAT013,2


In [ ]:
recomendacoes = []
# Criação do dicionário Disciplina_Liberam
Disciplina_Liberam = (
    Disciplinas_Requerimento[["DISCIPLINA", "Liberam"]]
    .set_index("DISCIPLINA")["Liberam"]
    .to_dict()
)

# Filtra as disciplinas não cursadas
disciplinas_obrigatorias_nao_cursadas = disciplinas_obrigatorias[
    ~disciplinas_obrigatorias["DISCIPLINA"].isin(usuario.materias_aprovadas)
]
disciplinas_optativas_nao_cursadas = disciplinas_optativas[
    ~disciplinas_optativas["DISCIPLINA"].isin(usuario.materias_aprovadas)
]


# Ordena as disciplinas obrigatórias do próximo semestre
disciplinas_obrigatorias_proximo = disciplinas_obrigatorias_nao_cursadas[
    disciplinas_obrigatorias_nao_cursadas["PER."] == usuario.semestre + 1
]
disciplinas_obrigatorias_proximo[
    "Disciplina_Liberam"
] = disciplinas_obrigatorias_proximo["CODIGO"].map(Disciplina_Liberam)
disciplinas_obrigatorias_proximo = disciplinas_obrigatorias_proximo.sort_values(
    by=["Disciplina_Liberam"], ascending=[False]
)

# Verifica disciplinas obrigatórias dos semestres anteriores, que não foram cursadas
disciplinas_obrigatorias_passados = disciplinas_obrigatorias_nao_cursadas[
    disciplinas_obrigatorias_nao_cursadas["PER."] < usuario.semestre
]
disciplinas_obrigatorias_passados[
    "Disciplina_Liberam"
] = disciplinas_obrigatorias_passados["CODIGO"].map(Disciplina_Liberam)
disciplinas_obrigatorias_passados = disciplinas_obrigatorias_passados.sort_values(
    by=["Disciplina_Liberam"], ascending=[False]
)

# recomendacao de obrigatorias
for _, row in disciplinas_obrigatorias_proximo.iterrows():
    if row["CODIGO"] not in usuario.materias_aprovadas and len(recomendacoes) < 5:
        recomendacoes.append(f'{row["CODIGO"]} - {row["DISCIPLINA"]}')

# recomendacao de obrigatorias dos semestres passados
# que por algum motivo nao cursei
for _, row in disciplinas_obrigatorias_passados.iterrows():
    if row["CODIGO"] not in usuario.materias_aprovadas and len(recomendacoes) < 5:
        recomendacoes.append(f'{row["CODIGO"]} - {row["DISCIPLINA"]}')

# Se não foram recomendadas 5 disciplinas, verificar disciplinas optativas
if len(recomendacoes) < 5:
    # Ordenação das disciplinas optativas não cursadas com base na carga horária (CH) e pré-requisitos
    disciplinas_optativas_nao_cursadas[
        "Disciplina_Liberam"
    ] = disciplinas_optativas_nao_cursadas["CODIGO"].map(Disciplina_Liberam)
    disciplinas_optativas_nao_cursadas = disciplinas_optativas_nao_cursadas.sort_values(
        by=["Disciplina_Liberam", "CH"], ascending=[False, True]
    )

    for _, row in disciplinas_optativas_nao_cursadas.iterrows():
        if (
            row["CODIGO"] not in usuario.materias_aprovadas
            and all(
                prereq in usuario.materias_aprovadas
                for prereq in row["PRE-REQUISITOS"].split(", ")
            )
            and len(recomendacoes) < 5
        ):
            recomendacoes.append(f'{row["CODIGO"]} - {row["DISCIPLINA"]}')

# Impressão das recomendações e semestre atual
print("Recomendações para o próximo semestre:")
for recom in recomendacoes:
    print(recom)

print("Semestre que estou cursando:", usuario.semestre)

Recomendações para o próximo semestre:
SPAD02 - Banco de Dados II
SPAD03 - Introdução à Análise de Dados
XRSC01 - Redes de Computadores
ADM51E - Gestão do Conhecimento
XDES01 - Fundamentos de Programação
Semestre que estou cursando: 4


C:\Users\lucas\AppData\Local\Temp\ipykernel_15600\1871796780.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disciplinas_obrigatorias_proximo[
C:\Users\lucas\AppData\Local\Temp\ipykernel_15600\1871796780.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disciplinas_obrigatorias_passados[


**Explicações da logica** 

Combina obrigatórias e optativas em um DataFrame

In [ ]:
disciplinas = pd.concat(
    [disciplinas_obrigatorias, disciplinas_optativas], ignore_index=True
)


# usando um comando que veio ao importar o pandas, é possivel unir dataframes


# pd.concat([Dataframe1, Dataframe2]) -> o ignore_index é somente para literalmente juntar os 2 dataframes de forma que não ligue para o index

Procurar as disciplinas obrigatórias e optativas que não foram cursadas


In [ ]:
disciplinas_obrigatorias_nao_cursadas = disciplinas_obrigatorias[
    ~disciplinas_obrigatorias["DISCIPLINA"].isin(Disciplinas_Cursadas)
]


# disciplinas_obrigatorias['DISCIPLINA'].isin(Disciplinas_Cursadas)


# essa parte cria uma "serie booleana" que caso cada disciplinas obrigatoria esteja presente nas cursadas retorna true, caso contrario false.


# o "~" esta invertendo os valores, dessa forma estou pegando as disciplinas obrigatorias que NÃO foram cursadas.


# o resultado disso armazeno na Disciplina_obrigatorias_nao_cursadas


# a mesma lógica se aplica para as materias optativas!!

NameError: name 'Disciplinas_Cursadas' is not defined

Ordenar as disciplinas obrigatórias do próximo semestre

In [ ]:
disciplinas_obrigatorias_proximo = disciplinas_obrigatorias_nao_cursadas[
    disciplinas_obrigatorias_nao_cursadas["PER."] == usuario.semestre + 1
]


# aqui estou friltrando as disciplinas obrigatorias que ainda não foram cursadas para o proximo semestre (semestre_atual + 1)


# dessa forma ao comparar [PER.] == semestre_atual+1 estou pegando apenas as disciplinas do proximo semestre.


disciplinas_obrigatorias_proximo[
    "Quantidade_Liberada"
] = disciplinas_obrigatorias_proximo["CODIGO"].map(Disciplina_Liberam)


# Cria uma coluna chamada 'Disciplina_Liberam' no dataframe


# Na coluna insiro valores que resultam da aplicação de 'Disciplinas_liberam' em cada elemento da coluna 'CODIGO'


disciplinas_obrigatorias_proximo = disciplinas_obrigatorias_proximo.sort_values(
    by=["Quantidade_Liberada"], ascending=[False]
)


# ordeno as disciplinas com base na 'Quantidade_liberada' em ordem decrescente


# as disciplinas com maior quantidade liberada vão estar em cima


recomendacoes = [
    f'{row["CODIGO"]} - {row["DISCIPLINA"]}'
    for _, row in disciplinas_obrigatorias_proximo.iterrows()
]


# Aqui estou usando o dataframe disciplinas_obrigatorias_proximo


# para cada linha gera uma string contendo o código e o nome da disciplina.


# Essas strings são armazenadas na lista recomendacoes.


# OBS: O ' _ ' indica que não estou usando o valor do índice. O 'row' representa os dados da linha atual em cada iteração.

C:\Users\bruno\AppData\Local\Temp\ipykernel_14448\46377514.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disciplinas_obrigatorias_proximo['Quantidade_Liberada'] = disciplinas_obrigatorias_proximo['CODIGO'].map(Disciplina_Liberam)


Verificar disciplinas obrigatórias dos semestres anteriores que o aluno deixou de cursar

In [ ]:
disciplinas_obrigatorias_passados = disciplinas_obrigatorias_nao_cursadas[
    disciplinas_obrigatorias_nao_cursadas["PER."] < usuario.semestre
]


# Aqui seleciono as disciplinas obrigatórias que não foram cursadas até o semestre atual.


# usando a condição de que o valor na coluna 'PER.' seja menor que o valor de semestre_atual


# assim faço com que o disciplinas_obrigatorias_passados receba somente as disciplinas que eu não fiz dos semestres passados


disciplinas_obrigatorias_passados[
    "Disciplina_Liberam"
] = disciplinas_obrigatorias_passados["CODIGO"].map(Disciplina_Liberam)


# Acessa a coluna 'CODIGO' no disciplinas_obrigatorias_passados e faz retornar os valores que representam os códigos das disciplinas obrigatórias passadas.


# para cada código de disciplina, o valor correspondente no dict Disciplina_Liberam será atribuído à nova coluna 'Disciplina_Liberam'.


# em resumo cria uma nova coluna que mostra a quantidade de disciplinas que cada disciplina obrigatória passada pode liberar


# esse "passada" diz respeito as materias dos semestres passados


disciplinas_obrigatorias_passados = disciplinas_obrigatorias_passados.sort_values(
    by=["Disciplina_Liberam"], ascending=[False]
)


# Ordenar as o dataframe por Disciplina_liberam (pontos criticos aparecem primeiro)

C:\Users\bruno\AppData\Local\Temp\ipykernel_14448\554352038.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disciplinas_obrigatorias_passados['Disciplina_Liberam'] = disciplinas_obrigatorias_passados['CODIGO'].map(Disciplina_Liberam)


Recomendacao das Disciplinas

In [ ]:
for _, row in disciplinas_obrigatorias_passados.iterrows():
    if row["CODIGO"] not in usuario.materias_aprovadas and len(recomendacoes) < 5:
        recomendacoes.append(f'{row["CODIGO"]} - {row["DISCIPLINA"]}')
# Para cada linha em disciplina obrigatória passada, verifica se o código da disciplina não está na lista de disciplinas já cursadas
# e se o número de recomendações é menor que 5
# adiciona o codigo e a disciplina na recomendacao

if len(recomendacoes) < 5:
    disciplinas_optativas_nao_cursadas[
        "Disciplina_Liberam"
    ] = disciplinas_optativas_nao_cursadas["CODIGO"].map(Disciplina_Liberam)
    disciplinas_optativas_nao_cursadas = disciplinas_optativas_nao_cursadas.sort_values(
        by=["Disciplina_Liberam", "CH"], ascending=[False, True]
    )

    for _, row in disciplinas_optativas_nao_cursadas.iterrows():
        if (
            row["CODIGO"] not in usuario.materias_aprovadas
            and all(
                prereq in usuario.materias_aprovadas
                for prereq in row["PRE-REQUISITOS"].split(", ")
            )
            and len(recomendacoes) < 5
        ):
            recomendacoes.append(f'{row["CODIGO"]} - {row["DISCIPLINA"]}')

# Se o número de recomendações for menor que 5 após as recomendações das disciplinas obrigatórias,
# ordena as disciplinas optativas não cursadas por Disciplina_Liberam(pontos criticos aparecem primeiro) e na carga horária,
# para cada disciplina optativa, verifica se o código não está na lista de disciplinas já cursadas,
# se todos os pré-requisitos estão nas disciplinas cursadas e se o número de recomendações é menor que 5,
# adiciona o codigo e a disciplina na recomendacao